# Scrape images with EasyICD

## Required imports

In [3]:
from easy_icd.scraping.scrape_images import scrape_images

## Define search terms, class names, and image directory

In [5]:
class_names = ['shark', 'dog', 'horse', 'deer']
search_terms = [['great white shark', 'hammerhead shark', 'basking shark'],
                ['german shepherd', 'afghan hound', 'golden retriever', 'labrador'],
               ['stallion', 'white horse'],
                ['deer', 'canadian moose', 'elk']]
images_per_class = [[5, 5, 5], [3, 3, 6, 3], [10, 5], [5, 5, 5]]
image_dir = 'scraping_example_images'

scrape_images(class_names, image_dir, search_terms, images_per_class)

{'shark': {'great white shark': {'successful': True,
   'num_desired_images': 5,
   'num_saved_images': 5,
   'time_taken_in_seconds': 6.78118634223938},
  'hammerhead shark': {'successful': True,
   'num_desired_images': 5,
   'num_saved_images': 5,
   'time_taken_in_seconds': 3.6699790954589844},
  'basking shark': {'successful': True,
   'num_desired_images': 5,
   'num_saved_images': 5,
   'time_taken_in_seconds': 3.450639486312866},
  'time_taken_in_seconds': 13.90180492401123,
  'num_desired_images': 15,
  'num_saved_images': 15,
  'successful': True},
 'dog': {'german shepherd': {'successful': True,
   'num_desired_images': 3,
   'num_saved_images': 3,
   'time_taken_in_seconds': 3.808958053588867},
  'afghan hound': {'successful': True,
   'num_desired_images': 3,
   'num_saved_images': 3,
   'time_taken_in_seconds': 6.226581811904907},
  'golden retriever': {'successful': True,
   'num_desired_images': 6,
   'num_saved_images': 6,
   'time_taken_in_seconds': 3.841588258743286}